# IERG 5350 Assignment 1: Tabular Reinforcement Learning

*2020-2021 Term 1, IERG 5350: Reinforcement Learning. Department of Information Engineering, The Chinese University of Hong Kong. Course Instructor: Professor ZHOU Bolei. Assignment author: PENG Zhenghao, SUN Hao, ZHAN Xiaohang.*


| Student Name | Student ID |
| :----: | :----: |
| Anglepsw | Tonystark |

------

Welcome to the assignment 1 of our RL course. The objective of this assignment is for you to understand the classic methods used in tabular reinforcement learning. 

This assignment has the following sections:

 - Section 1: Warm-up on the RL environment (35 points)
 - Section 2: Implementation of model-based family of algorithms: policy iteration and value iteration. (65 points)

You need to go through this self-contained notebook, which contains **21 TODOs** in part of the cells and has special `[TODO]` signs. You need to finish all TODOs. Some of them may be easy such as uncommenting a line, some of them may be difficult such as implementing a function. You can find them by searching the `[TODO]` symbol. However, we suggest you to go through the documents step by step, which will give you a better sense of the content.

You are encouraged to add more code on extra cells at the end of the each section to investigate the problems you think interesting. At the end of the file, we left a place for you to optionaly write comments (Yes, please give us some either negative or positive rewards so we can keep improving the assignment!). 

Please report any code bugs to us via **github issues**.

Before you get start, remember to follow the instruction at https://github.com/cuhkrlcourse/ierg5350-assignment to setup your environment.

Now start running the cells sequentially (by `ctrl + enter` or `shift + enter`) to avoid unnecessary errors by skipping some cells. 


## Section 1: Warm-up on the RL environment

(35/100 points)

In this section, we will go through the basic concepts of RL environments using OpenAI Gym. Besides, you will get the first sense of the toy environment we will use in the rest of the assignment.

Every Gym environment should contain the following attributes:

1. `env.step(action)` To step the environment by applying `action`. Will return four things: `observation, reward, done, info`, wherein `done` is a boolean value indicating whether this **episode** is finished. `info` may contain some information the user is interested in, we do not use it.
2. `env.reset()` To reset the environment, back to the initial state. Will return the initial observation.
3. `env.render()` To render the current state of the environment for human-being
4. `env.action_space` The allowed action format. In our case, it is `Discrete(4)` which means the action is an integer in the range [0, 1, 2, 3]. Therefore the `action` for `step(action)` should obey the limit of the action space.
5. `env.observation_space` The observation space.
6. `env.seed(seed)` To set the random seed of the environment. So the result is replicable.

Note that the word **episode** means the process that an agent interacts with the environment from the initial state to the terminal state. Within one episode, the agent will only receive one `done=True`, when it goes to the terminal state (the agent is dead or the game is over).

We will use "FrozenLake8x8-v0" as our environment. In this environment, the agent controls the movement of a character in a grid world. Some tiles of the grid are walkable, and others lead to the agent falling into the water. Additionally, the movement direction of the agent is uncertain and only partially depends on the chosen direction. The agent is rewarded for finding a walkable path to a goal tile. The meaning of each character:

1. S : starting point, safe
2. F : frozen surface, safe
3. H : hole, fall to your doom
4. G : goal, where the frisbee is located


In [1]:
# Run this cell without modification

# Import some packages that we need to use
from utils import *
import gym
import numpy as np
from collections import deque

### Section 1.1: Make the environment

You need to know 

1. How to make an environment
2. How to set the random seed of environment
3. What is observation space and action space

In [2]:
# Solve the TODOs and remove `pass`

# [TODO] Just a reminder. Do you add your name and student 
# ID in the table at top of the notebook?

# Create the environment
env = gym.make('FrozenLake8x8-v0')

# You need to reset the environment immediately after instantiating env. 
env.reset()  # [TODO] uncomment this line

# Seed the environment
env.seed(0)  # [TODO] uncomment this line

print("Current observation space: {}".format(env.observation_space))
print("Current action space: {}".format(env.action_space))
print("0 in action space? {}".format(env.action_space.contains(0)))
print("5 in action space? {}".format(env.action_space.contains(5)))
print("3 in action space? {}".format(env.action_space.contains(3)))
print(type(env.action_space))

Current observation space: Discrete(64)
Current action space: Discrete(4)
0 in action space? True
5 in action space? False
3 in action space? True
<class 'gym.spaces.discrete.Discrete'>


### Section 1.2: Play the environment with random actions

You need to know 

1. How to step the environment
2. How to render the environment

In [3]:
# Solve the TODOs and remove `pass`

# Run 1000 steps for test, terminate if done.
# You can run this cell multiples times.
env.reset()

while True:
    # take random action
    # [TODO] Uncomment next line
    obs, reward, done, info = env.step(env.action_space.sample())

    # render the environment
    env.render()  # [TODO] Uncomment this line

    print("Current observation: {}\nCurrent reward: {}\n"
          "Whether we are done: {}\ninfo: {}".format(
        obs, reward, done, info
    ))
    wait(sleep=0.2)

    # [TODO] terminate the loop if done
    if done:
        break


  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
Current observation: 29
Current reward: 0.0
Whether we are done: True
info: {'prob': 0.3333333333333333}


### Section 1.3: Define the evaluation function to value the random baseline

Now we need to define an evaluation function to evaluate a given policy (a function where the input is observation and the output is action). This is convenient for future evaluation.

As a reminder, you should create a `FrozenLake8x8-v0` environment instance by default, reset it after each episode (and at the beginning), step the environment, and terminate episode if done.

After implementing the `evaluate` function, run the next cell to check whether you are right.

In [4]:
# Solve the TODOs and remove `pass`

def _render_helper(env):
    env.render()
    wait(sleep=0.2)


def evaluate(policy, num_episodes, seed=0, env_name='FrozenLake8x8-v0', render=False):
    """[TODO] You need to implement this function by yourself. It
    evaluate the given policy and return the mean episode reward.
    We use `seed` argument for testing purpose.
    You should pass the tests in the next cell.

    :param policy: a function whose input is an interger (observation)
    :param num_episodes: number of episodes you wish to run
    :param seed: an interger, used for testing.
    :param env_name: the name of the environment
    :param render: a boolean flag. If true, please call _render_helper
    function.
    :return: the averaged episode reward of the given policy.
    """

    # Create environment (according to env_name, we will use env other than 'FrozenLake8x8-v0')
    env = gym.make(env_name)

    # Seed the environment
    env.seed(seed)
    
    # Build inner loop to run.
    # For each episode, do not set the limit.
    # Only terminate episode (reset environment) when done = True.
    # The episode reward is the sum of all rewards happen within one episode.
    # Call the helper function `render(env)` to render
    rewards = []
    for i in range(num_episodes):
        # reset the environment
        obs = env.reset()
        act = policy(obs)
        
        ep_reward = 0
        while True:
            # [TODO] run the environment and terminate it if done, collect the
            # reward at each step and sum them to the episode reward.
            obs, reward, done, info = env.step(act)
            ep_reward=ep_reward+reward
            act = policy(obs)
            if render:
                _render_helper(env)
            if done:
                break
        
        rewards.append(ep_reward)

    return np.mean(rewards)

# [TODO] Run next cell to test your implementation!

In [5]:
# Run this cell without modification

# Run this cell to test the correctness of your implementation of `evaluate`.
LEFT = 0
DOWN = 1
RIGHT = 2
UP = 3

def expert(obs):
    """Go down if agent at the right edge, otherwise go right."""
    return DOWN if (obs + 1) % 8 == 0 else RIGHT

def assert_equal(seed, value, env_name):
    ret = evaluate(expert, 1000, seed, env_name=env_name)
    assert ret == value, \
    "When evaluate on seed {}, 1000 episodes, in {} environment, the " \
    "averaged reward should be {}. But you get {}." \
    "".format(seed, env_name, value, ret)
      
assert_equal(0, 0.065, 'FrozenLake8x8-v0')
assert_equal(1, 0.059, 'FrozenLake8x8-v0')
assert_equal(2, 0.055, 'FrozenLake8x8-v0')

assert_equal(0, 0.026, 'FrozenLake-v0')
assert_equal(1, 0.034, 'FrozenLake-v0')
assert_equal(2, 0.028, 'FrozenLake-v0')

print("Test Passed!")
print("\nAs a baseline, the mean episode reward of a hand-craft "
      "agent is: ", evaluate(expert, 1000))

Test Passed!

As a baseline, the mean episode reward of a hand-craft agent is:  0.065


Congraduation! You have finished section 1 (if and only if not error happen at the above codes).

If you want to do more investigation, feel free to open new cells via `Esc + B` after the next cells and write codes in it, so that you can reuse some result in this notebook. Remember to write sufficient comments and documents to let others know what you are doing.

In [6]:
# You can do more inverstigation here if you wish. Leave it blank if you don't.
print(env.observation_space.contains(63))
amali=np.zeros((2,2))
amali[0,1]

True


0.0

------

## Section 2: Model-based Tabular RL

(65/100 points)

We have learned how to use the Gym environment to run an episode, as well as how to interact between the agent (policy) and environment via `env.step(action)` to collect observation, reward, done, and possible extra information.

Now we need to build the basic tabular RL algorithm to solve this environment. **Note that compared to the model-free methods in the Sec.3, the algorithms in this section needs to access the internal information of the environment, namely the transition dynamics**. In our case, given a state and an action, we need to know which state current environment would jump to, and the probability of this happens, and the reward if the transition happens. You will see that we provide you a helper function `trainer._get_transitions(state, action)` that takes state and action as input and return you a list of possible transitions.

You will use a class to represent a Trainer, which seems to be over-complex for tabular RL. But we will use the same framework in the future assignments, or even in your future research. So it would be helpful for you to get familiar with how to implement an RL algorithm in a class-orientetd programming style, as a first step toward the implementation of state of the art RL algorithm in the future.

In [7]:
# Run this cell without modification

class TabularRLTrainerAbstract:
    """This is the abstract class for tabular RL trainer. We will inherent the specify 
    algorithm's trainer from this abstract class, so that we can reuse the codes like
    getting the dynamic of the environment (self._get_transitions()) or rendering the
    learned policy (self.render())."""
    
    def __init__(self, env_name='FrozenLake8x8-v0', model_based=True):
        self.env_name = env_name
        self.env = gym.make(self.env_name)
        self.action_dim = self.env.action_space.n
        self.obs_dim = self.env.observation_space.n
        
        self.model_based = model_based

    def _get_transitions(self, state, act):
        """Query the environment to get the transition probability,
        reward, the next state, and done given a pair of state and action.
        We implement this function for you. But you need to know the 
        return format of this function.
        """
        self._check_env_name()
        assert self.model_based, "You should not use _get_transitions in " \
            "model-free algorithm!"
        
        # call the internal attribute of the environments.
        # `transitions` is a list contain all possible next states and the 
        # probability, reward, and termination indicater corresponding to it
        transitions = self.env.env.P[state][act]

        # Given a certain state and action pair, it is possible
        # to find there exist multiple transitions, since the 
        # environment is not deterministic.
        # You need to know the return format of this function: a list of dicts
        ret = []
        for prob, next_state, reward, done in transitions:
            ret.append({
                "prob": prob,
                "next_state": next_state,
                "reward": reward,
                "done": done
            })
        return ret
    
    def _check_env_name(self):
        assert self.env_name.startswith('FrozenLake')

    def print_table(self):
        """print beautiful table, only work for FrozenLake8X8-v0 env. We 
        write this function for you."""
        self._check_env_name()
        print_table(self.table)

    def train(self):
        """Conduct one iteration of learning."""
        raise NotImplementedError("You need to override the "
                                  "Trainer.train() function.")

    def evaluate(self):
        """Use the function you write to evaluate current policy.
        Return the mean episode reward of 1000 episodes when seed=0."""
        result = evaluate(self.policy, 1000, env_name=self.env_name)
        return result

    def render(self):
        """Reuse your evaluate function, render current policy 
        for one episode when seed=0"""
        evaluate(self.policy, 1, render=True, env_name=self.env_name)

### Section 2.1: Policy Iteration

Recall the idea of policy iteration: 

1. Update the state value function, given all possible transitions in the environment.
2. Find the best policy that makes the most out of the current state value function.
3. If the best policy is identical to the previous one then stop the training. Otherwise, go to step 1.

In step 1, the way to update the state value function is by 

$$v_{k+1} = E_{s'}[r(s, a)+\gamma v_{k}(s')]$$

wherein the $a$ is given by current policy, $s'$ is next state, $r$ is the reward, $v_{k}(s')$ is the next state value given by the old (not updated yet) value function. The expectation is computed among all possible transitions (given a state and action pair, it is possible to have many different next states, since the environment is not deterministic).

In step 2, the best policy is to take the action with maximum expected return given a state:

$$a = {argmax}_a E_{s'}[r(s, a) + \gamma v_{k}(s')]$$

Policy iteration algorithm has an outer loop (update policy, step 1 to 3) and an inner loop (fit the value function, within step 1). In each outer loop, we call once `trainer.train()`, where we call `trainer.update_value_function()` once to update the value function (the state value table). After that we call `trainer.update_policy()` to update the current policy. `trainer` object has a `trainer.policy` attribute, which is a function that takes observation as input and returns an action.

You should implement the trainer following the framework we already wrote for you. Please carefully go through the codes and finish all `TODO` in it.

In [31]:
# Solve the TODOs and remove `pass`

class PolicyItertaionTrainer(TabularRLTrainerAbstract):
    def __init__(self, gamma=1.0, eps=1e-10, env_name='FrozenLake8x8-v0'):
        super(PolicyItertaionTrainer, self).__init__(env_name)

        # discount factor
        self.gamma = gamma

        # value function convergence criterion
        self.eps = eps

        # build the value table for each possible observation
        self.table = np.zeros((self.obs_dim,))

        # [TODO] you need to implement a random policy at the beginning.
        # It is a function that take an integer (state or say observation)
        # as input and return an interger (action).
        # remember, you can use self.action_dim to get the dimension (range)
        # of the action, which is an integer in range
        # [0, ..., self.action_dim - 1]
        # hint: generating random action at each call of policy may lead to
        #  failure of convergence, try generate random actions at initializtion
        #  and fix it during the training.
        #self.policy = lambda obs: np.random.randint(0,self.action_dim)
        self.policy=None
        #policy_table = np.random.choice(self.action_dim,[self.obs_dim, ])
        policy_table = np.random.randint(0,self.action_dim,size=(self.obs_dim,))
        self.policy = lambda state:policy_table[state]

        # test your random policy
        test_random_policy(self.policy, self.env)

    def train(self):
        """Conduct one iteration of learning."""
        # [TODO] value function may be need to be reset to zeros.
        # if you think it should, than do it. If not, then move on.
        # hint: the value function is equivalent to self.table,
        #  a numpy array with length 64.
        self.table=np.zeros((self.obs_dim,))

        self.update_value_function()
        self.update_policy()

    def update_value_function(self):
        count = 0  # count the steps of value updates
        while True:
            old_table = self.table.copy()

            for state in range(self.obs_dim):#遍历每一个状态
                act = self.policy(state)
                transition_list = self._get_transitions(state, act)#得到状态转移表，注意是字典
                
                state_value = 0
                for transition in transition_list:
                    prob = transition['prob']
                    reward = transition['reward']
                    next_state = transition['next_state']
                    done = transition['done']
                    
                    # [TODO] what is the right state value?
                    # hint: you should use reward, self.gamma, old_table, prob,
                    # and next_state to compute the state value
                    state_value=state_value+(reward+self.gamma*old_table[next_state])*prob

                # update the state value
                self.table[state] = state_value

            # [TODO] Compare the old_table and current table to
            #  decide whether to break the value update process.
            # hint: you should use self.eps, old_table and self.table
            should_break = None
            if np.linalg.norm(self.table-old_table)<=self.eps:
            #if np.sum(np.abs(old_table - self.table))<=self.eps:   
                should_break=True

            if should_break:
                break
            count += 1
            if count % 200 == 0:
                # disable this part if you think debug message annoying.
                print("[DEBUG]\tUpdated values for {} steps. "
                      "Difference between new and old table is: {}".format(
                    count, np.sum(np.abs(old_table - self.table))
                ))
            if count > 4000:
                print("[HINT] Are you sure your codes is OK? It shouldn't be "
                      "so hard to update the value function. You already "
                      "use {} steps to update value function within "
                      "single iteration.".format(count))
            if count > 6000:
                raise ValueError("Clearly your code has problem. Check it!")

    def update_policy(self):
        """You need to define a new policy function, given current
        value function. The best action for a given state is the one that
        has greatest expected return.

        To optimize computing efficiency, we introduce a policy table,
        which take state as index and return the action given a state.
        """
        policy_table = np.zeros([self.obs_dim, ], dtype=np.int)

        for state in range(self.obs_dim):
            state_action_values = [0] * self.action_dim

            # [TODO] assign the action with greatest "value"
            # to policy_table[state]
            # hint: what is the proper "value" here?
            #  you should use table, gamma, reward, prob,
            #  next_state and self._get_transitions() function
            #  as what we done at self.update_value_function()
            #  Bellman equation may help.
            best_action = None
            for action in range(self.action_dim):
                transition_list = self._get_transitions(state, action)#得到状态转移表，注意是字典
                
                state_value = 0
                for transition in transition_list:
                    prob = transition['prob']
                    reward = transition['reward']
                    next_state = transition['next_state']
                    done = transition['done']
                    state_value=state_value+(reward+self.gamma*self.table[next_state])*prob
                #if state_value>=state_action_values[0]:
                #    best_action=action
                #    state_action_values[0]=state_value
                state_action_values[action]=state_value
            best_action=np.argmax(state_action_values)
            
            policy_table[state] = best_action

        self.policy = lambda obs: policy_table[obs]


Now we have built the Trainer class for policy iteration algorithm. In the following few cells, we will train the agent to solve the problem and evaluate its performance.

In [32]:
# Solve the TODOs and remove `pass`

# Managing configurations of your experiments is important for your research.
default_pi_config = dict(
    max_iteration=1000,
    evaluate_interval=1,
    gamma=1.0,
    eps=1e-10
)


def policy_iteration(train_config=None):
    config = default_pi_config.copy()
    if train_config is not None:
        config.update(train_config)
        
    trainer = PolicyItertaionTrainer(gamma=config['gamma'], eps=config['eps'])

    old_policy_result = {
        obs: -1 for obs in range(trainer.obs_dim)
    } #这个的输出结果是啥

    for i in range(config['max_iteration']):
        # train the agent
        trainer.train()  # [TODO] please uncomment this line

        # [TODO] compare the new policy with old policy to check whether
        #  should we stop. If new and old policy have same output given any
        #  observation, them we consider the algorithm is converged and
        #  should be stopped.
        should_stop = None
        new_policy_result={obs:trainer.policy(obs) for obs in range(trainer.obs_dim)}
        if new_policy_result==old_policy_result:
            should_stop=True
        
        if should_stop:
            print("We found policy is not changed anymore at "
                  "itertaion {}. Current mean episode reward "
                  "is {}. Stop training.".format(i, trainer.evaluate()))
            break
        old_policy_result = new_policy_result

        # evaluate the result
        if i % config['evaluate_interval'] == 0:
            print(
                "[INFO]\tIn {} iteration, current mean episode reward is {}."
                "".format(i, trainer.evaluate()))

            if i > 20:
                print("You sure your codes is OK? It shouldn't take so many "
                      "({}) iterations to train a policy iteration "
                      "agent.".format(i))

    assert trainer.evaluate() > 0.8, \
        "We expect to get the mean episode reward greater than 0.8. " \
        "But you get: {}. Please check your codes.".format(trainer.evaluate())

    return trainer


In [33]:
# Run this cell without modification

# It may be confusing to call a trainer agent. But that's what we normally do.
pi_agent = policy_iteration()

[INFO]	In 0 iteration, current mean episode reward is 0.0.
[INFO]	In 1 iteration, current mean episode reward is 0.0.
[DEBUG]	Updated values for 200 steps. Difference between new and old table is: 0.01064319560233583
[DEBUG]	Updated values for 400 steps. Difference between new and old table is: 0.0005654325301605277
[DEBUG]	Updated values for 600 steps. Difference between new and old table is: 3.0039281256930006e-05
[DEBUG]	Updated values for 800 steps. Difference between new and old table is: 1.595872841673139e-06
[DEBUG]	Updated values for 1000 steps. Difference between new and old table is: 8.478265914498018e-08
[DEBUG]	Updated values for 1200 steps. Difference between new and old table is: 4.5041800532386045e-09
[INFO]	In 2 iteration, current mean episode reward is 0.0.
[DEBUG]	Updated values for 200 steps. Difference between new and old table is: 0.016743973761202728
[DEBUG]	Updated values for 400 steps. Difference between new and old table is: 0.0008895436862262698
[DEBUG]	Update

In [34]:
# Run this cell without modification

print("Your policy iteration agent achieve {} mean episode reward. The optimal score "
      "should be almost {}.".format(pi_agent.evaluate(), 0.86))

Your policy iteration agent achieve 0.867 mean episode reward. The optimal score should be almost 0.86.


In [35]:
# Run this cell without modification

pi_agent.render()

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [36]:
# Run this cell without modification

pi_agent.print_table()

+-----+-----+-----State Value Mapping-----+-----+-----+
|     |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |
|-----+-----+-----+-----+-----+-----+-----+-----+-----|
| 0   |1.000|1.000|1.000|1.000|1.000|1.000|1.000|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 1   |1.000|1.000|1.000|1.000|1.000|1.000|1.000|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 2   |1.000|0.978|0.926|0.000|0.857|0.946|0.982|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 3   |1.000|0.935|0.801|0.475|0.624|0.000|0.945|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 4   |1.000|0.826|0.542|0.000|0.539|0.611|0.852|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----

Congratulations! You have successfully implemented the policy iteration trainer (if and only if no error happens at the above cells). Few further problems for you to investigate:

1. What is the impact of the discount factor gamma?
2. What is the impact of the value function convergence criterion epsilon?

If you are interested in doing more investigation (not limited to these two), feel free to open new cells at the end of this notebook and left a clear trace of your thinking and coding, which leads to extra credit if you do a good job. It's an optional job, and you can ignore it.

Now let's continue our journey!

### Section 2.2: Value Iteration

Recall the idea of value iteration. We update the state value: 

$$v_{k+1}(s) = \max_a E_{s'} [r(s, a) + \gamma v_{k}(s')]$$

wherein the $s'$ is next state, $r$ is the reward, $v_{k}(s')$ is the next state value given by the old (not updated yet) value function. The expectation is computed among all possible transitions (given a state and action pair, it is possible to have many different next states, since the environment is not deterministic).

The value iteration algorithm does not require an inner loop. It computes the expected return of all possible actions at a given state and uses the maximum of them as the state value. You can imagine it "pretends" we already have the optimal policy and run policy iteration based on it. Therefore we do not need to maintain a policy object in a trainer. We only need to retrieve the optimal policy using the same rule as policy iteration, given current value function.

You should implement the trainer following the framework we already wrote for you. Please carefully go through the code and finish all `TODO` in it.

In [87]:
# Solve the TODOs and remove `pass`


class ValueIterationTrainer(PolicyItertaionTrainer):
    """Note that we inherate Policy Iteration Trainer, to resue the
    code of update_policy(). It's same since it get optimal policy from
    current state-value table (self.table).
    """

    def __init__(self, gamma=1.0, env_name='FrozenLake8x8-v0'):
        super(ValueIterationTrainer, self).__init__(gamma, None, env_name)

    def train(self):
        """Conduct one iteration of learning."""
        # [TODO] value function may be need to be reset to zeros.
        # if you think it should, than do it. If not, then move on.
        #pass
        #self.table=np.zeros((self.obs_dim,)) #加上这一句结果就不对，为什么呢？因为这个不是初始化函数啊！！！
        print("当前值表格为{}".format(type(self.table)))
        print("当前值表格shape为{}".format((self.table.shape)))
        print("当前值表格为\n{}".format((self.table)))

        # In value iteration, we do not explicit require a
        # policy instance to run. We update value function
        # directly based on the transitions. Therefore, we
        # don't need to run self.update_policy() in each step.
        self.update_value_function()

    def update_value_function(self):
        old_table = self.table.copy()

        for state in range(self.obs_dim):
            state_value = 0

            # [TODO] what should be de right state value?
            # hint: try to compute the state_action_values first
            #pass
            for action in range(self.action_dim):
                transition_list = self._get_transitions(state, action)#得到状态转移表，注意是字典
                
                action_value_temp = 0
                for transition in transition_list:
                    prob = transition['prob']
                    reward = transition['reward']
                    next_state = transition['next_state']
                    done = transition['done']
                    action_value_temp+=(reward+self.gamma*old_table[next_state])*prob
                if action_value_temp>=state_value:
                    state_value=action_value_temp
                #print("state_value={}".format(state_value))

            self.table[state] = state_value

        # Till now the one step value update is finished.
        # You can see that we do not use a inner loop to update
        # the value function like what we did in policy iteration.
        # This is because to compute the state value, which is
        # a expectation among all possible action given by a
        # specified policy, we **pretend** already own the optimal
        # policy (the max operation).

    def evaluate(self):
        """Since in value itertaion we do not maintain a policy function,
        so we need to retrieve it when we need it."""
        self.update_policy()
        return super().evaluate()

    def render(self):
        """Since in value itertaion we do not maintain a policy function,
        so we need to retrieve it when we need it."""
        self.update_policy()
        return super().render()


In [88]:
# Solve the TODOs and remove `pass`

# Managing configurations of your experiments is important for your research.
default_vi_config = dict(
    max_iteration=10000,
    evaluate_interval=100,  # don't need to update policy each iteration
    gamma=1.0,
    eps=1e-10
)


def value_iteration(train_config=None):
    config = default_vi_config.copy()
    if train_config is not None:
        config.update(train_config)

    # [TODO] initialize Value Iteration Trainer. Remember to pass
    #  config['gamma'] to it.
    #pass
    trainer=ValueIterationTrainer(gamma=config['gamma'])
    old_policy_result = {
        obs: -1 for obs in range(trainer.obs_dim)
    } #这个的输出结果是啥

    old_state_value_table = trainer.table.copy()

    for i in range(config['max_iteration']):
        # train the agent
        trainer.train()  # [TODO] please uncomment this line

        # evaluate the result
        if i % config['evaluate_interval'] == 0:
            print("[INFO]\tIn {} iteration, current "
                  "mean episode reward is {}.".format(
                i, trainer.evaluate()
            ))

            # [TODO] compare the new policy with old policy to check should
            #  we stop.
            # [HINT] If new and old policy have same output given any
            #  observation, them we consider the algorithm is converged and
            #  should be stopped.
            should_stop = None
            #pass
            trainer.update_policy()
            new_policy_result={obs:trainer.policy(obs) for obs in range(trainer.obs_dim)}
            if new_policy_result==old_policy_result:
                should_stop=True
            old_policy_result=new_policy_result
            #for obs in range(trainer.obs_dim):
            #    if trainer.table[obs]!=old_state_value_table[obs]:
            #        break
            #    should_stop=True
            #old_state_value_table = trainer.table.copy()
            
            if should_stop:
                print("We found policy is not changed anymore at "
                      "itertaion {}. Current mean episode reward "
                      "is {}. Stop training.".format(i, trainer.evaluate()))
                break

            if i > 3000:
                print("You sure your codes is OK? It shouldn't take so many "
                      "({}) iterations to train a policy iteration "
                      "agent.".format(
                    i))

    assert trainer.evaluate() > 0.8, \
        "We expect to get the mean episode reward greater than 0.8. " \
        "But you get: {}. Please check your codes.".format(trainer.evaluate())

    return trainer


In [89]:
# Run this cell without modification

vi_agent = value_iteration()

当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[INFO]	In 0 iteration, current mean episode reward is 0.0.
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.33333333 0.         0.         0.         0.
 0.         0.         0.33333333 0.        ]
当

[INFO]	In 100 iteration, current mean episode reward is 0.892.
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.64602455 0.66167066 0.68415996 0.70848874 0.73270853 0.75511839
 0.77281709 0.77767287 0.64211161 0.65404795 0.67361149 0.69692037
 0.72185026 0.74795519 0.77770391 0.7873213  0.61988986 0.61372749
 0.58272268 0.         0.62238314 0.71422444 0.78230485 0.80608664
 0.57722693 0.55041866 0.47523063 0.30314328 0.44108849 0.
 0.77174402 0.83334172 0.50621364 0.44963784 0.30538893 0.
 0.40534805 0.48817397 0.70717757 0.86798766 0.45144422 0.
 0.         0.11841571 0.29242275 0.35729002 0.         0.90854164
 0.41378913 0.         0.07777706 0.06475553 0.         0.29489274
 0.         0.95321378 0.39459365 0.27622567 0.17235552 0.
 0.26421015 0.52899311 0.76421015 0.        ]
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.65123992 0.66662619 0.6887534  0.71270588 0.73655906 0.75863022
 0.77606463 0.78088902 0.64739471 0.65913125 0.6783761  0.70131683
 0.72586136

[INFO]	In 200 iteration, current mean episode reward is 0.867.
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.91441209 0.91794402 0.92306785 0.92869642 0.93442648 0.93996623
 0.94497324 0.94867263 0.91353504 0.9162225  0.92065411 0.92597525
 0.93168347 0.93754624 0.94366562 0.95091589 0.89544266 0.88098146
 0.83484514 0.         0.79805095 0.88889093 0.9335062  0.95530435
 0.86594653 0.81700287 0.7063706  0.42686486 0.57601397 0.
 0.90363139 0.96164623 0.84107925 0.708225   0.47068737 0.
 0.50501553 0.58070182 0.81750959 0.96966435 0.82167328 0.
 0.         0.15419476 0.35970387 0.42083253 0.         0.97900828
 0.80837147 0.         0.15662111 0.1034009  0.         0.32295685
 0.         0.98926966 0.80161033 0.58214416 0.36776135 0.
 0.2741425  0.54841863 0.7741425  0.        ]
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.9155894  0.91907812 0.92413946 0.92969938 0.93535873 0.94082857
 0.9457705  0.94942039 0.91472321 0.91737772 0.9217552  0.92701134
 0.93264932

[INFO]	In 300 iteration, current mean episode reward is 0.867.
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.9798181  0.98069903 0.98197154 0.98335899 0.98475818 0.98609658
 0.98729343 0.98816949 0.97959837 0.98026914 0.98137131 0.98268661
 0.98408562 0.98550722 0.98697264 0.98868654 0.96846075 0.94956381
 0.90101054 0.         0.84243185 0.93257891 0.97037926 0.98969805
 0.95857988 0.89873787 0.77308338 0.46126295 0.61123293 0.
 0.93495157 0.99115979 0.95027423 0.78940918 0.52056054 0.
 0.53050737 0.60357557 0.84372846 0.9930079  0.94380717 0.
 0.         0.16415571 0.37707399 0.43680643 0.         0.99516159
 0.93938115 0.         0.18266434 0.11554208 0.         0.3299933
 0.         0.99752675 0.93713347 0.68416253 0.43287729 0.
 0.27661865 0.55327199 0.77661865 0.        ]
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.98011174 0.9809799  0.98223395 0.98360126 0.98498012 0.98629907
 0.98747852 0.98834184 0.9798952  0.98055624 0.98164243 0.98293864
 0.98431734 

[INFO]	In 400 iteration, current mean episode reward is 0.867.
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.99534602 0.99554942 0.99584318 0.99616343 0.9964863  0.99679508
 0.99707114 0.99727317 0.99529528 0.99545016 0.99570461 0.99600822
 0.99633109 0.99665909 0.99699711 0.99739235 0.99198174 0.97072967
 0.91996339 0.         0.85323049 0.94296721 0.97915409 0.99762549
 0.98900155 0.92506873 0.79372706 0.47137338 0.62053956 0.
 0.94233337 0.99796241 0.98646886 0.81586486 0.53645204 0.
 0.53714527 0.60932081 0.84998574 0.99838838 0.9844802  0.
 0.         0.16701432 0.38166856 0.44091123 0.         0.99888479
 0.98311108 0.         0.19129438 0.11941645 0.         0.33180192
 0.         0.99942994 0.98241332 0.71824327 0.45460043 0.
 0.27725549 0.55451993 0.77725549 0.        ]
当前值表格为<class 'numpy.ndarray'>
当前值表格shape为(64,)
当前值表格为
[0.99541382 0.99561425 0.99590374 0.99621932 0.99653749 0.99684177
 0.99711381 0.9973129  0.99536382 0.99551644 0.99576719 0.99606638
 0.99638454

[INFO]	In 500 iteration, current mean episode reward is 0.867.
We found policy is not changed anymore at itertaion 500. Current mean episode reward is 0.867. Stop training.


In [90]:
# Run this cell without modification

print("Your value iteration agent achieve {} mean episode reward. The optimal score "
      "should be almost {}.".format(vi_agent.evaluate(), 0.86))

Your value iteration agent achieve 0.867 mean episode reward. The optimal score should be almost 0.86.


In [91]:
# Run this cell without modification

vi_agent.render()

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [92]:
# Run this cell without modification

vi_agent.print_table()

+-----+-----+-----State Value Mapping-----+-----+-----+
|     |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |
|-----+-----+-----+-----+-----+-----+-----+-----+-----|
| 0   |0.999|0.999|0.999|0.999|0.999|0.999|0.999|0.999|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 1   |0.999|0.999|0.999|0.999|0.999|0.999|0.999|0.999|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 2   |0.998|0.976|0.925|0.000|0.856|0.945|0.981|0.999|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 3   |0.997|0.932|0.799|0.474|0.623|0.000|0.944|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 4   |0.997|0.823|0.541|0.000|0.539|0.611|0.851|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----

Congratulation! You have successfully implemented the value iteration trainer (if and only if no error happens at the above cells). Few further problems for you to investigate:

1. Do you see that some iteration during training yields better rewards than the final one?  Why does that happen?
2. What is the impact of the discount factor gamma?
3. What is the impact of the value function convergence criterion epsilon?

If you are interested in doing more investigation (not limited to these two), feel free to open new cells at the end of this notebook and left a clear trace of your thinking and coding, which leads to extra credit if you do a good job. It's an optional job, and you can ignore it.

Now let's continue our journey!

### Section 2.3: Compare two model-based agents

Now we have two agents: `pi_agent` and `vi_agent`. They are believed to be the optimal policy in this environment. Can you compare the policy of two of them and use a clean and clear description or figures to show your conclusion?

In [93]:
# Solve the TODO and remove `pass`

# [TODO] try to compare two trained agents' policies
# hint: trainer.print_table() may give you a better sense.
#pass
pi_agent.print_table()
vi_agent.print_table()

+-----+-----+-----State Value Mapping-----+-----+-----+
|     |   0 |   1 |   2 |   3 |   4 |   5 |   6 |   7 |
|-----+-----+-----+-----+-----+-----+-----+-----+-----|
| 0   |1.000|1.000|1.000|1.000|1.000|1.000|1.000|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 1   |1.000|1.000|1.000|1.000|1.000|1.000|1.000|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 2   |1.000|0.978|0.926|0.000|0.857|0.946|0.982|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 3   |1.000|0.935|0.801|0.475|0.624|0.000|0.945|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
| 4   |1.000|0.826|0.542|0.000|0.539|0.611|0.852|1.000|
|     |     |     |     |     |     |     |     |     |
+-----+-----+-----+-----+-----+-----+-----+-----

In [ ]:
# You can do more inverstigation here if you wish. Leave it blank if you don't.


------

## Conclusion and Discussion

In this assignment, we learn how to use Gym package, how to use Object Oriented Programming idea to build a basic tabular RL algorithm.

It's OK to leave the following cells empty. In the next markdown cell, you can write whatever you like. Like the suggestion on the course, the confusing problems in the assignments, and so on.

If you want to do more investigation, feel free to open new cells via `Esc + B` after the next cells and write codes in it, so that you can reuse some result in this notebook. Remember to write sufficient comments and documents to let others know what you are doing.

Following the submission instruction in the assignment to submit your assignment to our staff. Thank you!

------

...